# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками

## Задание 1.

В однопроцессорную  систему случайным образом поступают на выполнение $m$ разных типов программ, отличающихся известной трудоемкостью $Q_1, Q_2, \ldots, Q_m$. Входящий поток простейший с интенсивностью $\lambda$.

- Представив данную систему как одноканальную СМО с неограниченной очередью, вычислить среднее время обслуживания программ, считая длительность обслуживания случайной величиной (теоретически и экспериментально).

- Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ. Сравнить полученные результаты.

- Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет величина кванта на среднее время обслуживания программ.

In [1]:
Variant<-8
set.seed(Variant) 
m<-sample(c(6:20),1)
lambda<-runif(1,0.1, 2)
Q<-rexp(m,0.3)
q<-sample(c(1:4),1)
View(data.frame(m,q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
9,3,1.61935


[1] 1.012475452 3.263946321 1.459516694 2.855469094 1.489090714 2.516819748
[7] 0.299747437 4.973630433 0.003007615


#Часть 1:

Заданная система является системой вида $M/M/1/\infty$.

Среднее время обслуживания для данного вида систем:

$$ T_{сист} = \frac{1}{\mu(1-\rho)} = \frac{1}{\mu-\lambda} $$

Определим интенсивность обслуживания $\mu$:

$$ \mu = \frac{1}{Q_{ср}} =  \frac{m}{\sum_{i=1}^{m}Q_i} $$



Проведем теоретические рассчеты в среде R

In [10]:
mu <- m / sum(Q)
mu
lambda

# lambda > mu, уменьшим лямбду
lambda <- lambda - 1.2

1 / (mu - lambda)

[1] 0.503533

[1] 1.61935

[1] 11.8789

Проведем экспериментальный расчет в среде R:

In [14]:
N <- 1000000    # Число тиков (прибавления дельт времени)
dt <- 0.01      # Тик (Дельта времени)
curTime <- 0    # Текущее время симуляции

q <- c()    # очередь заявок в систему
qDur <- 0   # суммарная длительность всех заявок в очереди
currentRequest <- 0 # текущий обрабатываемый запрос

numOfReqs <- 0
timeInSystTotal <- 0
receiveRequestAt <- rexp(1, lambda)

getHandleDuration <- function() {
  i <-sample(c(1:m),1)
  return(rexp(1, 1/{Q[i]}))
}

for (i in 1:N) {
  curTime <- curTime + dt

  if (curTime >= receiveRequestAt) {
    numOfReqs <- numOfReqs + 1
    handleDuration <- getHandleDuration()
    if (currentRequest == 0) {
      currentRequest <- handleDuration
      timeInSystTotal <- timeInSystTotal + handleDuration
    } else {
      q <- append(q, handleDuration)
      qDur <- qDur + handleDuration
      timeInSystTotal <- timeInSystTotal + qDur
    }
    receiveRequestAt <- receiveRequestAt + rexp(1, lambda)
  }

  # сокращение времени обработки текущей программы, которая находится на сервере.
  if (currentRequest >= 0) {
    currentRequest <- currentRequest - dt
    if (currentRequest <= 0) {
      if (length(q) > 0) {
        qDur <- qDur - q[1]
        currentRequest <- q[1]
        q <- q[-1]
      } else {
        currentRequest <- 0
      }
    }
  }
}

timeInSystTotal / numOfReqs

[1] 12.46537

#Часть 2:

